# gevent使用

### gevent简介

### gevent的一些坑

gevent的猴子补丁会替换掉标准库的一些函数,使网络变成异步非阻塞,通常在程序的一开始处就进行猴子补丁,否则有可能补丁没有生效

通常在程序一开始处执行`from gevent import monkey; monkey.patch_all()`

有的时候打了猴子补丁,会产生一些怪异的情况,这些怪异的情况使无法判别的,所以猴子补丁谨慎使用.

## gevent用法

gevent对协程的支持,本质上是greenlet在实现切换工作

### gevent使用流程

In [ ]:
# spawn方法可以看作是创建一个协程, joinall方法可以看作是添加任务,并启动运行协程
from gevent import monkey;monkey.patch_all()
import gevent
import requests


def run_task(url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)

        
if __name__ == '__main__':
    urls = ['url1', 'url2', 'url3']
    greenlets = [gevent.spawn(run_task, url) for url in urls]
    gevent.joinall(greenlets)

### 协程池的使用

In [ ]:
# -*- coding:utf-8 -*-
from gevent import monkey; monkey.patch_all()
import gevent
from gevent.pool import Pool

exist_pages = []
with open("xxxxx.txt", "r", encoding="utf-8") as f:
    for line in f:
        exist_pages.append(int(line.strip()))


def get_page_index():
    for page in range(1, 623000):
        yield page


def record_leak_pages(page):
    with open("xxxx.txt", "a", encoding="utf-8") as f:
        f.write("{}\n".format(page))


def check_page_exist(page):
    if page not in exist_pages:
        record_leak_pages(page)


pool = Pool(200)
try:
    pool.map(check_page_exist, get_page_index())
except Exception as e:
    print(e)